# learn_to_generate_articles

The purpose of this notebook is to try different systems to generate TED
stories from a given topic and related articles.

My current plan is to create some local JSON files with different
inputs for prompts. If there is time, we can set up a way to automatically
evaluate output quality, and iterate systematically. As a v1, I plan
to simply look at the results by hand and begin iterating that way.

## Imports

In [2]:
import json

import openai

## Load in API keys

You will need your own local `config.json` with the
format `{"api_key": <key>}`.

In [3]:
with open('config.json') as f:
    openai.api_key = json.load(f)['api_key']

## Create an example input

This section is meant to only be run once.

In [6]:
example_input = {'topic': 'The Future of French Architecture', 'articles': []}

In [7]:
fnames = ['architecture_of_paris.txt', 'french_architecture.txt', 'grand_palais.txt',
    'jean_nouvel.txt', 'paris_architecture_of_the_belle_epoque.txt']

In [8]:
for fname in fnames:
    with open(f'raw_articles/{fname}') as f:
        example_input['articles'].append(f.read())

In [13]:
with open('example_input_1_french_architecture.json', 'w') as f:
    json.dump(example_input, f)

## Load in an example dataset to learn with

The data has the following format:

    {
        "topic": <short topic string>,
        "articles": [list of article strings],
        "image_urls": [list of image urls]
    }

To begin with, I will not use image input. The
plan is to begin using that later.

In [17]:
with open('example_input_1_french_architecture.json') as f:
    example_input = json.load(f)

topic = example_input['topic']
articles = example_input['articles']

## Create a sample prompt

In [33]:
def make_prompt(topic, articles):
    
    prefix = f'''Below is a collection of information about {topic}.
        After that information I will ask you to write the text of a TED talk about it.
    '''
    suffix = f'''Using information you already know, in addition to the information above,
        please write the transcript of a TED talk about {topic}.
    '''

    # TODO: Switch to thinking in tokens instead of words.

    word_limit = 17_000  # This is a hack for the 32k token limit.

    # We could consider using a weighted limit per article; longer
    # articles can send in more tokens.
    max_words_per_article = word_limit / len(articles)

    parts = [prefix]
    for article in articles:
        start = ' '.join(article.split()[:int(max_words_per_article)])
        parts.append(start)
    parts.append(suffix)

    return '\n'.join(parts)

In [34]:
prompt = make_prompt(topic, articles)

In [35]:
print(len(prompt.split()))

15629


## Create a sample TED talk

In [36]:
NUM_REPLY_TOKENS = 8_000

In [37]:
def get_gpt_response(prompt):
    return openai.ChatCompletion.create(
        model = 'gpt-4-32k',
        messages = [
            {
                'role': 'user',
                'content': prompt
            }
        ],
        max_tokens = NUM_REPLY_TOKENS,
        top_p = 1.0,
        frequency_penalty = 0.0,
        presence_penalty = 0.0
    )

In [38]:
response = get_gpt_response(prompt)

In [46]:
print(response['choices'][0]['message']['content'])

Ladies and gentlemen, it's a pleasure to be here today to talk to you about a topic that is at once steeped in history, but full of potential for the future: the architecture of France. It's not just an exploration of aesthetics or engineering; French architecture offers a compelling narrative of the country's spirit, values, and evolution. As we venture to the future of French architecture, let's start by examining its storied past.

If you've visited Paris, you've seen firsthand the chronicles of architectural achievement, from the Middle Ages to the 21st century. Paris was the birthplace of the Gothic style, evident in the majestic Cathedral Notre-Dame de Paris, and later gave the world the Art Nouveau style, with its organic and flowing designs. The city has witnessed a dramatic interplay of styles that represented different eras: the French Renaissance, Classical revival, the exuberant Belle Époque, and innovative Art Deco. Landmarks such as the Grand Palais stand testament to thi

## Save this out as a sample transcript

In [47]:
save_obj = {
    'topic': topic,
    'transcript': response['choices'][0]['message']['content']
}

In [49]:
with open('example_transcript_1_french_architecture.json', 'w') as f:
    json.dump(save_obj, f)